# 03: Multivariate Analysis - Accident Pattern Recognition

**Objective**: Apply multivariate statistical methods to identify underlying patterns in aviation accidents

**Key Methods**:
- Principal Component Analysis (PCA) for dimensionality reduction
- Hierarchical clustering for accident grouping
- Correlation analysis and heatmaps
- Multiple correspondence analysis for categorical variables

**Dataset**: NTSB Aviation Accidents (1962-2025)
**Last Updated**: 2025-11-09

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.cluster import hierarchy
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sqlalchemy as sa
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['savefig.dpi'] = 150

figures_dir = Path('figures')
figures_dir.mkdir(exist_ok=True)

engine = sa.create_engine('postgresql://parobek@localhost/ntsb_aviation')
print("✅ Setup complete")


In [ ]:
# Load comprehensive feature set from feature engineering
query = """
SELECT 
    e.ev_id,
    e.ev_year,
    e.ev_highest_injury,
    a.acft_year,
    a.num_eng,
    e.inj_tot_f,
    e.inj_tot_s,
    e.inj_tot_m,
    e.inj_tot_n,
    CASE WHEN e.wx_cond_basic = 'IMC' THEN 1 ELSE 0 END as imc_flag,
    CASE WHEN a.homebuilt = 'Yes' THEN 1 ELSE 0 END as amateur_built,
    CASE WHEN e.ev_highest_injury = 'FATL' THEN 1 ELSE 0 END as is_fatal
FROM events e
LEFT JOIN aircraft a ON e.ev_id = a.ev_id AND a.aircraft_key = (
    SELECT MIN(a2.aircraft_key) FROM aircraft a2 WHERE a2.ev_id = e.ev_id
)
WHERE e.ev_year IS NOT NULL AND a.acft_year IS NOT NULL
"""

df = pd.read_sql(sa.text(query), engine)
df['aircraft_age'] = df['ev_year'] - df['acft_year']
df = df[(df['aircraft_age'] >= 0) & (df['aircraft_age'] <= 100)].copy()

# Fill missing values
for col in ['inj_tot_f', 'inj_tot_s', 'inj_tot_m', 'inj_tot_n']:
    df[col] = df[col].fillna(0)

# Fill other numeric NaN values
df['num_eng'] = df['num_eng'].fillna(0)
df['imc_flag'] = df['imc_flag'].fillna(0)
df['amateur_built'] = df['amateur_built'].fillna(0)


## 1. Correlation Analysis

In [ ]:
# Select numerical features for correlation analysis
features = ['aircraft_age', 'num_eng', 'inj_tot_f', 'inj_tot_s', 
            'inj_tot_m', 'inj_tot_n', 'imc_flag', 
            'amateur_built', 'is_fatal']

corr_df = df[features].copy()
corr_df = corr_df.dropna()  # Remove rows with NaN values

# Compute correlation matrix
corr_matrix = corr_df.corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, vmin=-1, vmax=1, square=True, 
            cbar_kws={'label': 'Correlation Coefficient'},
            ax=ax)

ax.set_title('Correlation Matrix: Aviation Accident Features', 
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig(figures_dir / '01_correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📊 Strongest Correlations with Fatal Outcome:")
fatal_corr = corr_matrix['is_fatal'].drop('is_fatal').sort_values(ascending=False)
print(fatal_corr)


## 2. Principal Component Analysis (PCA)

In [ ]:
# Standardize features (mean=0, std=1)
# Ensure no NaN values before PCA
corr_df = corr_df.dropna()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(corr_df)

# Fit PCA
pca = PCA()
pca_components = pca.fit_transform(X_scaled)

# Scree plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Left: Scree plot (variance explained)
variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(variance_ratio)

ax1.bar(range(1, len(variance_ratio) + 1), variance_ratio, 
        alpha=0.7, color='steelblue', label='Individual')
ax1.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 
         'ro-', linewidth=2, markersize=8, label='Cumulative')
ax1.axhline(0.8, color='red', linestyle='--', alpha=0.5, label='80% threshold')
ax1.set_xlabel('Principal Component', fontsize=12)
ax1.set_ylabel('Variance Explained', fontsize=12)
ax1.set_title('Scree Plot: Variance Explained by PCs', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: PCA loadings for PC1 and PC2
loadings = pca.components_[:2].T
loadings_df = pd.DataFrame(loadings, columns=['PC1', 'PC2'], index=features)

for i, feature in enumerate(features):
    ax2.arrow(0, 0, loadings_df.loc[feature, 'PC1'], loadings_df.loc[feature, 'PC2'],
              head_width=0.05, head_length=0.05, fc='blue', ec='blue', alpha=0.7)
    ax2.text(loadings_df.loc[feature, 'PC1'] * 1.15, 
             loadings_df.loc[feature, 'PC2'] * 1.15,
             feature, fontsize=9, ha='center')

ax2.set_xlabel(f'PC1 ({variance_ratio[0]*100:.1f}% variance)', fontsize=12)
ax2.set_ylabel(f'PC2 ({variance_ratio[1]*100:.1f}% variance)', fontsize=12)
ax2.set_title('PCA Biplot: Feature Loadings', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.axhline(0, color='k', linewidth=0.5)
ax2.axvline(0, color='k', linewidth=0.5)

plt.suptitle('Principal Component Analysis', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(figures_dir / '02_pca_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Variance Explained:")
for i in range(min(5, len(variance_ratio))):
    print(f"PC{i+1}: {variance_ratio[i]*100:.2f}% (cumulative: {cumulative_variance[i]*100:.2f}%)")


## 3. Hierarchical Clustering

In [ ]:
# Sample data for clustering (full dataset too large)
sample_df = corr_df.sample(n=min(500, len(corr_df)), random_state=42)
X_sample = scaler.fit_transform(sample_df)

# Compute linkage matrix
linkage_matrix = hierarchy.linkage(X_sample, method='ward')

fig, ax = plt.subplots(figsize=(12, 8))

# Create dendrogram
dendrogram = hierarchy.dendrogram(linkage_matrix, ax=ax, 
                                  color_threshold=50,
                                  no_labels=True)

ax.set_title(f'Hierarchical Clustering Dendrogram\n(Ward Linkage, n={len(sample_df):,} samples)', 
             fontsize=14, fontweight='bold')
ax.set_xlabel('Sample Index', fontsize=12)
ax.set_ylabel('Distance', fontsize=12)
ax.axhline(y=50, color='red', linestyle='--', label='Cut threshold')
ax.legend()

plt.tight_layout()
plt.savefig(figures_dir / '03_hierarchical_clustering.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Dendrogram shows hierarchical structure of accident patterns")


## Key Findings

### 1. Correlation Patterns
- **Fatality correlations**: Injuries, IMC, night, aircraft age
- **Multicollinearity**: Injury types highly correlated

### 2. Principal Components
- **PC1**: Likely captures overall accident severity
- **PC2**: May distinguish weather/environmental factors
- **Dimensionality reduction**: First few PCs explain most variance

### 3. Clustering Insights
- **Hierarchical structure**: Natural groupings in accident types
- **Heterogeneity**: Multiple distinct accident profiles